In [2]:
# auto-reload changed source files when they are imported
%reload_ext autoreload
%autoreload 2

# add top repo dir to path so that src can be imported
import sys
sys.path.append("..")

%matplotlib inline

In [6]:
import os.path as osp

import torch
from torch.nn import Linear, ReLU, MSELoss

from torch_geometric.nn import SchNet, Sequential
from torch_geometric.loader import DataLoader
from src.data.dataset import Crystals

**Model**

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# TODO: Dimensions for lin layers
model = Sequential(
    'z, pos',
    [
        (SchNet(),'z, pos -> z'),
        # Linear(),
        # ReLU(inplace=True),
        # Linear()
    ]
).to(device)

**Init**

In [19]:
save_path = '../data/processed/v1.pt'
dataset = Crystals(save_path)
dataset[0]


Data(x=[6, 5], edge_index=[2, 30], pos=[6, 3], element=[6], z=[6], dist=[30], y=[50])

In [21]:
train_dataset, val_dataset, test_dataset = dataset.get_splits(deterministic=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = MSELoss()

loader = DataLoader(test_dataset, batch_size=64)

**Train loop**

In [24]:

model.train()
for data in loader:
    data = data.to(device)
    pred = model(data.z, data.pos)

    # TODO: make sure pred and data.y have same dim and have corresponding elements
    loss = loss_fn(pred, data.y)
    
    loss.backward()
    optimizer.step()
    print(pred)
    print(pred.shape)


RuntimeError: expected scalar type Float but found Double